In [2]:
import sys
sys.path.append('../')

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
%reload_ext autoreload

In [5]:
from scripts.packages.detect_voids import get_neightbours, search_voids_bb_neightbours
from scripts.packages.yolo_predict import YOLO_Pred
from concurrent.futures import ThreadPoolExecutor
import pandas as pd
import cv2
import multiprocessing
from functools import partial
import itertools
import os

In [6]:
yolo = YOLO_Pred('/home/cremerf/FinalProject/data/first_training/weights/bestnoft.onnx', '/home/cremerf/FinalProject/data/config_blmodel.yaml')

In [7]:
img_path = '/home/cremerf/FinalProject/eudes-fede/test_imgs/test_7.jpg'

In [8]:
image = cv2.imread(img_path)
h_image, w_image = image.shape[0:2] # limits of the image
df_predictions = yolo.predictions(image=image)

# Get neightbours from 3 ways (right / left / up)
pool = multiprocessing.Pool()
neightbour = 'left'
func = partial(get_neightbours, df_predictions, neightbour)
dict_of_neightbours_left = pool.map(func, list(df_predictions.index))
pool.close()
pool.join()

# clean up empty positions
dict_of_neightbours_left = list(filter(None, dict_of_neightbours_left))

pool = multiprocessing.Pool()
neightbour = 'right'
func = partial(get_neightbours, df_predictions, neightbour)
dict_of_neightbours_right = pool.map(func, list(df_predictions.index))
pool.close()
pool.join()

dict_of_neightbours_right = list(filter(None, dict_of_neightbours_right))

# Merged 3 separated list of dicts(left/right/high) into 1 list
dicts_neightbours = [dict_of_neightbours_left, dict_of_neightbours_right]
list_neightbours = list(itertools.chain.from_iterable(dicts_neightbours))

# Get void neightbours 
list_of_voids = search_voids_bb_neightbours(df_predictions=df_predictions, list_of_dicts=list_neightbours, h_image=h_image, w_image=w_image, img_path=img_path)

# Create dataframe with data(X_center/Y_center/Label) of voids
df_voids = pd.DataFrame(list_of_voids, columns=['Neightbour','Label','x1','y1','x2','y2', 'Width','Height'])

In [86]:
image = cv2.imread(img_path)

In [ ]:
x1 = int(df_predictions.loc[1233]['x1']) 
y1 = int(df_predictions.loc[1233]['y1']) 
x2 = int(df_predictions.loc[1233]['x2']) 
y2 = int(df_predictions.loc[1233]['y2']) 

In [99]:
#x1 = int(df_predictions.loc[int(1233)][0] - df_predictions.loc[int(1233)][2]/2) 
#y1 = int(df_predictions.loc[int(1233)][1] - df_predictions.loc[int(1233)][3]/2) 
#x2 = int(df_predictions.loc[int(1233)][0] + df_predictions.loc[int(1233)][2]/2)  
#y2 = int(df_predictions.loc[int(1233)][1] + df_predictions.loc[int(1233)][3]/2)  

In [54]:
df_voids.head(5)

,Neightbour,Label,x1,y1,x2,y2,Width,Height
0,393,393 void #1,1575.464939,2290.024667,1770.959368,2357.109402,195.494429,67.084735
1,391,391 void #1,1449.175397,2291.150212,1622.804498,2356.895906,173.629102,65.745694
2,420,420 void #1,1259.791166,2424.219517,1455.223125,2511.737850,195.431960,87.518334
3,768,768 void #1,1090.320600,2728.377701,1292.464675,2806.587802,202.144075,78.210100
4,89,89 void #1,1766.191532,2258.931440,1916.023254,2394.208763,149.831721,135.277323


In [34]:
df_voids

,Neightbour,Label,x1,y1,x2,y2,Width,Height
0,393,393 void #1,1575.464939,2290.024667,1770.959368,2357.109402,195.494429,67.084735
1,391,391 void #1,1449.175397,2291.150212,1622.804498,2356.895906,173.629102,65.745694
2,420,420 void #1,1259.791166,2424.219517,1455.223125,2511.737850,195.431960,87.518334
3,768,768 void #1,1090.320600,2728.377701,1292.464675,2806.587802,202.144075,78.210100
4,89,89 void #1,1766.191532,2258.931440,1916.023254,2394.208763,149.831721,135.277323
5,561,561 void #1,587.345615,1382.521900,696.249605,1750.651769,108.903990,368.129869
6,562,562 void #1,665.838094,1388.161055,766.640690,1749.020958,100.802596,360.859902
7,943,943 void #1,1926.983928,2240.429894,2059.756581,2385.724574,132.772653,145.294680
8,965,965 void #1,2049.975786,2381.535893,2157.981736,2523.983962,108.005949,142.448070
9,600,600 void #1,910.415169,1465.027548,1017.968460,1765.828678,107.553291,300.801130


In [13]:
#image_path_bb = os.path.join(folder_path_final, filename)

img_path = '/home/cremerf/FinalProject/scripts/test_neightbours2.jpg'
# load the image
image = cv2.imread(img_path)

# loop over indexes and for each index plot the rectangles
#for idx in df_voids.index:

# get the coordinates for each index/rectangle
for i in df_voids.index:
    x1 = int(df_voids.loc[i][2]) 
    y1 = int(df_voids.loc[i][3]) 
    x2 = int(df_voids.loc[i][4])  
    y2 = int(df_voids.loc[i][5])


    # # Window name in which image is displayed
    window_name = 'Object'
    # represents the top left corner of rectangle
    start_point=(x1, y1)

    # represents the top right corner of rectangle
    end_point=(x2,y2)

    # # Blue color in BGR
    color = (0, 0, 255)

    # # Line thickness of 2 px
    thickness = 5

    # plot the rectangle over the image
    image = cv2.rectangle(image, start_point, end_point, color, thickness)

# save the img
cv2.imwrite(filename='test_voids8.jpg', img=image)

True

In [37]:
img_path = '/home/cremerf/FinalProject/data/images/test/test_0.jpg'

In [39]:
df_predictions.head(5)

,X_center,Y_center,Width,Height
572,1130.014352,1566.114157,119.923841,347.048751
1059,1073.070552,1188.697504,125.808713,359.978320
581,1603.940973,1564.580640,83.080911,310.984321
585,1751.918793,1564.916821,83.728849,308.152010
1203,759.620471,2645.523212,138.472683,139.737216


In [66]:
def compute_iou(contour, bbox):
  # Import the necessary packages
  import cv2

  # Compute the bounding box of the contour
  contour_bbox = cv2.boundingRect(contour)

  bbox_array = np.array(bbox)

  # Compute the intersection and union of the contour and bounding box
  intersection = contour_bbox and bbox_array
  union = contour_bbox | bbox_array

  # Compute the intersection over union (IoU)
  iou = (intersection / union)

  # Return the IoU
  return iou

In [42]:
image = cv2.imread('/home/cremerf/FinalProject/scripts/test_neightbours2.jpg')
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

In [67]:
# Import the necessary packages
import numpy as np
import cv2

# Load the image and convert it to grayscale
image = cv2.imread('/home/cremerf/FinalProject/scripts/test_neightbours2.jpg')
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

# Define the minimum IoU value
min_iou = 0.1

# Loop over all bounding boxes in the dataframe
for index, row in df_predictions.iterrows():
    # Get the bounding box coordinates from the dataframe
    x_center = row['X_center']
    y_center = row['Y_center']
    w = int(row['Width'])
    h = int(row['Height'])

    # Compute the bounding box coordinates from the center and dimensions
    x = int(x_center - w / 2)
    y = int(y_center - h / 2)

    # Extract the region of interest
    roi = gray[y:y + h, x:x + w]

    # Compute the binary image by thresholding the region of interest
    ret, binary = cv2.threshold(roi, 128, 255, cv2.THRESH_BINARY_INV)

    # Find the contours in the binary image
    contours, _ = cv2.findContours(binary, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)

    # Loop over the contours
    for contour in contours:
      # Compute the intersection over union (IoU) between the contour and the bounding box
      iou = compute_iou(contour, (x, y, w, h))

      # If the IoU is larger than the minimum IoU, draw the contour on the image
      if iou > min_iou:
        cv2.drawContours(image, [contour], -1, (0, 255, 0), 2)

# Show the output image
cv2.imwrite("Image.jpg", image)

ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()

In [67]:
h_image, w_image = image.shape[0:2]

In [73]:
image = cv2.imread(img_path)

for id in df_voids.index:
    text = f'{df_voids.iloc[id][1]}'

    w = int(df_voids.iloc[id][6] * x_factor)
    h = int(df_voids.iloc[id][7] * y_factor)
    x = int(df_voids.iloc[id][2] + (w / 2) * x_factor)
    y = int(df_voids.iloc[id][5] - (h / 2) * y_factor)

    cv2.rectangle(image,(x,y),(x+w,y+h),2)
    cv2.rectangle(image,(x,y-30),(x+w,y),-1)

    cv2.putText(image,str(id),(x,y-10),cv2.FONT_HERSHEY_PLAIN,0.7,(0,0,0),1)

cv2.imwrite(filename='test_voids.jpg', img=image)

True

In [41]:
data = pd.read_csv('/home/cremerf/FinalProject/data_downloaded/annotations_train.csv', names=["image_name", "x1", "y1", "x2", "y2","class", "image_width", "image_height" ])

In [43]:
image.shape

(3264, 2448, 3)

In [80]:
df_voids.head(5)

,Neightbour,Label,x1,y1,x2,y2,Width,Height
0,1233,1233 void #1,51.607074,538.354998,73.610795,564.667219,22.003721,26.312222
1,234,234 void #1,33.589325,189.115864,64.682465,264.807995,31.093140,75.692131
2,92,92 void #1,384.399222,436.086927,405.496396,464.358995,21.097174,28.272068
3,791,791 void #1,36.381460,103.366776,61.432955,166.861404,25.051495,63.494629
4,59,59 void #1,428.310845,405.009939,440.533404,426.178720,12.222559,21.168781
